In [1]:
from typing import Dict, TypedDict
from langgraph.graph import StateGraph, END
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
import pandas as pd
from IPython.display import display, Image
from langchain_core.runnables.graph import MermaidDrawMethod
from dotenv import load_dotenv
import os
import uuid
from pydantic import BaseModel, Field
from typing import List, TypedDict
from urllib.request import urlopen 
import json 
# import praw
# from prawcore.exceptions import ResponseException
import requests
from datetime import datetime, timedelta
from langgraph.graph import StateGraph, START,END
from langgraph.checkpoint.memory import MemorySaver
from dotenv import load_dotenv

In [ ]:
load_dotenv()
os.environ['GROQ_API_KEY'] = ''
from langchain_groq import ChatGroq
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [3]:
from typing import TypedDict

class HubspotAgent(TypedDict):
  requirememt: str
  deal_price: str
  
  initial_greet: str
  
  
  qualified:str
  polite_reply: str
  price_ask_str: str
  price: int
  scheduler: str
  assign: str
  esclator:str

In [4]:
def inital_greet(state:HubspotAgent):
    
    prompt = f'''
    I am a CTO at a AI consulting company named as 'Elint Data' and I have a lead on hubspot for which i want to draft an initial email stating we do the following services

    1) Agentic AI
    2) AI Red Teaming
    3)DevSecOps
    4)  High-Performance Computing

    I dont know much about thier company
    '''
        
    response = llm.invoke(prompt)
    
    return {'initial_greet':response}

In [5]:
def qualifier(state:HubspotAgent):
    text = state['requirememt']
    prompt = f'''you are a sales agent. you have a requirement from a customer. you need to qualify that can we do the required task.
    
    we are a software company. we are providing 4 services.
    1. Agentic AI
    2. AI Red Teaming
    3. DevSecOps
    4. High-Performance Computing
    
    
    
    the requirement is :{text}
    reply with just yes or no.
    '''
    
    response = llm.invoke(prompt)
    response = response.lower()
    
    return {'qualified':response}

In [6]:
def second_stage_reply(state:HubspotAgent):
    quali = state['qualified']
    if quali == 'yes':
        prompt = f'''you are a sales agent. you have to politely reply to a customer.
        That we can do the required project and we will assign a team to work on it.
        '''
        response1 = llm.invoke(prompt)
        return {'polite_reply':response1,'price_ask_str':'None'}
    else:
        prompt = f'''you are a sales agent. you have to politely reply to a customer.
        to ask for their budget of the project.
        '''
        response2 = llm.invoke(prompt)
        return {'price_ask_str':response2,'polite_reply':'None'}


In [7]:
def deal_price_extractor(state:HubspotAgent):
    text = state['deal_price']
    prompt = f'''
    you are a sales agent. you have to extract the price from the customer.
    the customer has given a deal price of {text}
    '''
    
    response = llm.invoke(prompt)
    response = response.lower()
    
    return {'scheduler':response}

In [8]:
def schedule(state:HubspotAgent):


    prompt = f'''
    you are a sales agent. you have to schedule a meeting with the customer.
    for the requirement of {state['requirememt']} for tomorrow at 10 am.
    '''
    
    response = llm.invoke(prompt)
    
    return {'price':response}

In [9]:
def assigner(state:HubspotAgent):
    prompt = f'''
    you are a sales agent. you have to assign a new project to a team member chirag.
    for the requirement of {state['requirememt']} and a meeting has been set for tomorrow at 10 am with the client.
    also ask chirag if he is can take up the project or not.
    '''
    response = llm.invoke(prompt)
    
    return {'assign':response}

In [21]:
from IPython.display import display, Image as PILImage

workflow = StateGraph(HubspotAgent)


workflow.add_node("inital_greet", inital_greet)
workflow.add_node("qualifier", qualifier)
workflow.add_node("second_stage_reply", second_stage_reply)
workflow.add_node("deal_price_extractor", deal_price_extractor)
workflow.add_node("schedule", schedule) 
workflow.add_node("assigner", assigner) 


workflow.set_entry_point("inital_greet")
workflow.add_edge("inital_greet", "qualifier")
workflow.add_edge("qualifier", "second_stage_reply")
workflow.add_edge("qualifier", END)

workflow.add_edge("second_stage_reply", "deal_price_extractor") 
workflow.add_edge("deal_price_extractor", "schedule") 
workflow.add_edge("deal_price_extractor", "assigner")
workflow.add_edge("schedule", END)
workflow.add_edge("assigner", END)



memory = MemorySaver()

graph_plan = workflow.compile(checkpointer=memory)
# display(Image(graph_plan.get_graph(xray=1).draw_mermaid_png()))

In [22]:
print(graph_plan.get_graph().draw_mermaid())


---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	inital_greet(inital_greet)
	qualifier(qualifier)
	second_stage_reply(second_stage_reply)
	deal_price_extractor(deal_price_extractor)
	schedule(schedule)
	assigner(assigner)
	__end__([<p>__end__</p>]):::last
	__start__ --> inital_greet;
	deal_price_extractor --> assigner;
	deal_price_extractor --> schedule;
	inital_greet --> qualifier;
	qualifier --> second_stage_reply;
	second_stage_reply --> deal_price_extractor;
	assigner --> __end__;
	schedule --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [15]:
# Execute the compiled workflow graph with a sample state
sample_state = HubspotAgent(
    requirememt="Develop an AI-based chatbot",
    deal_price="5000 USD",
    initial_greet="",
    qualified="",
    polite_reply="",
    price_ask_str="",
    price=0,
    scheduler="",
    assign="",
    esclator=""
)

result = graph_plan.run(sample_state)
print(result)

AttributeError: 'CompiledStateGraph' object has no attribute 'run'